In [2]:
from requestWithHandlingHttperr import requestWithHandlingHttperr
import json
import pandas as pd

In [3]:
league_list = []

In [4]:
headers = {
        "x-api-key" : "0TvQnueqKa5mxJntVWt0w4LpLfEkrV1Ta8rQBb9Z",
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.75 Safari/537.36'
    }
response = requestWithHandlingHttperr("https://prod-relapi.ewp.gg/persisted/gw/getLeagues?hl=en-US", headers = headers)
json = response.json()
if 'data' in json and len(json['data']) > 0:
    leagues = json["data"]["leagues"]
    for league in leagues:
        league_list.append(str(league["id"]))

In [5]:
tournament_list = []
for league_id in league_list:
    response = requestWithHandlingHttperr(f"https://prod-relapi.ewp.gg/persisted/gw/getTournamentsForLeague?hl=en-US&leagueId={league_id}", headers=headers)
    json = response.json()
    if "data" in json and len(json["data"]) > 0:
        tournaments = json["data"]["leagues"][0]["tournaments"]
        for tournament in tournaments:
            startDate = 0
            if len(tournament["startDate"]) > 4:
                startDate = int(tournament["startDate"][:4])
            if startDate >= 2023:
                tournament_list.append(tournament)

df = pd.DataFrame(tournament_list)
df

,id,slug,startDate,endDate
0,112523529392628088,lcs_summer_2024,2024-06-15,2024-09-07
1,111504625283627681,lcs_spring_2024,2024-01-19,2024-04-01
2,110303581083678395,lcs_summer_2023,2023-05-31,2023-08-22
3,109517090066605615,lcs_spring_2023,2023-01-24,2023-04-11
4,112352881163915249,lec_summer_2024,2024-06-08,2024-07-28
...,...,...,...,...
120,110372454557117766,lrs_closing_2023,2023-05-31,2023-08-28
121,110988896196596544,wqs_emea_na_2023,2023-10-08,2023-10-09
122,111102042176368993,latam_ddr_2023,2023-10-14,2023-10-16
123,111573236201487481,tft_events_2024,2023-12-31,2024-12-31


In [9]:
df["id"]

0      112523529392628088
1      111504625283627681
2      110303581083678395
3      109517090066605615
4      112352881163915249
              ...        
120    110372454557117766
121    110988896196596544
122    111102042176368993
123    111573236201487481
124    109761195185432372
Name: id, Length: 125, dtype: object

In [10]:
major_leagues = pd.read_excel("../data/pentakill 경기 상세데이터 수집기록.xlsx", sheet_name= "경기 세부 링크", dtype = "str")["id"]

In [11]:
major_leagues

0      112099354464733424
1      110198981276611770
2      107905284983470149
3      105873410870441926
4      109383739753133748
              ...        
184    110416959491275651
185    109749565997987636
186    109067014926424947
187    109066101880117809
188    107256578722475733
Name: id, Length: 189, dtype: object

In [16]:
df_minor = df[df["id"].isin(major_leagues) == False].reset_index(drop=True)
df_minor.to_excel("../data/target_tournament_for_test.xlsx")